In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd

# Линейный классификатор, логистическая регрессия
Построить модель логистической регрессии, которая предсказывает уровень дохода.

При возможности попробуйте улучшить точность предсказаний (метод score) с помощью перебора признаков.

Целевая переменная — уровень дохода income (крайний правый столбец). [Описание признаков](http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html).

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('./adult.csv')
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
num_features = ['age', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'gender', 'race']
columns_to_select = num_features + cat_features
X = pd.get_dummies(df[columns_to_select], cat_features)

In [5]:
income = LabelEncoder()
income.fit(df.income)
y = pd.Series(income.transform(df.income))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [8]:
model.score(X_test, y_test)

0.8523902139420616

### Пробуем XGBoost

In [9]:
from hyperopt import pyll, hp, fmin, tpe, STATUS_OK, Trials
import pprint
pp = pprint.PrettyPrinter(indent=4, width=100)

In [10]:
import xgboost as xgb

In [11]:
params = {'n_estimators': 50,
          'learning_rate': 0.6,
          'max_depth': 5,
          'min_child_weight': 1,
          'subsample': 1,
          'colsample_bytree': 0.5,
          'n_jobs': 8}

clf_xgb = xgb.XGBClassifier(**params)
clf_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.6, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=8, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [12]:
clf_xgb.score(X_test, y_test)

0.87634353567407108